## Dense Net

 - Training Dataset:  CutMix, beta=1, cutmix_prob=1
 - Sagemaker Notebook must be of type, conda_pytorch_p36
 
#### Install Requirements

In [5]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

In [6]:
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


#### Train the Model

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/densenet.yaml \
    model.densenet.depth 100 \
    model.densenet.growth_rate 12 \
    train.batch_size 64 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_CM_1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00 \
    scheduler.epochs 400

[2020-07-11 01:20:41] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_CM_1
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: densenet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
 

#### Refine the Model

In [7]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr 0.001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR

[2020-07-13 11:37:43] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: densenet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_

In [8]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00_resume400_50/test_results_0050_cifar10

[2020-07-13 21:41:29] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.08it/s]
[2020-07-13 21:41:50] __main__ INFO: Elapsed 19.44
[2020-07-13 21:41:50] __main__ INFO: Loss 0.3269 Accuracy 0.9369


In [9]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00_resume400_50/test_results_0050_cifar101

[2020-07-13 21:42:06] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.66it/s]
[2020-07-13 21:42:10] __main__ INFO: Elapsed 4.18
[2020-07-13 21:42:10] __main__ INFO: Loss 0.7624 Accuracy 0.8645


In [10]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00/test_results_0400_cifar10

[2020-07-13 21:42:46] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.08it/s]
[2020-07-13 21:43:07] __main__ INFO: Elapsed 19.44
[2020-07-13 21:43:07] __main__ INFO: Loss 0.4507 Accuracy 0.9055


In [11]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00/checkpoint_00400.pth \
   dataset.name CIFAR101 \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00/test_results_0400_cifar101

[2020-07-13 21:43:35] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.67it/s]
[2020-07-13 21:43:40] __main__ INFO: Elapsed 4.17
[2020-07-13 21:43:40] __main__ INFO: Loss 0.8656 Accuracy 0.8250


#### Record the Results

In [3]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['densenet_BC_100_12', 'densenet_BC_100_12', 'densenet_BC_100_12', 'densenet_BC_100_12'],
           'Testset': ['cifar10', 'cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 200, 300, 300],
           'Loss': [0.3681, 0.2279, 0.2223, 0.5098],
           'Accuracy': [0.8875, 0.9456, 0.9484, 0.8830],
           'Original_Accuracy': [95.5, 95.5, 95.5, 87.6],
           'Original_CI': [(95.1, 95.9), (95.1, 95.9), (95.1, 95.9), (86.1, 89.0)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])



#df.to_csv('/home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,densenet_BC_100_12,cifar10,100,0.3681,0.8875,95.5,"(95.1, 95.9)"
1,densenet_BC_100_12,cifar10,200,0.2279,0.9456,95.5,"(95.1, 95.9)"
2,densenet_BC_100_12,cifar10,300,0.2223,0.9484,95.5,"(95.1, 95.9)"
3,densenet_BC_100_12,cifar10.1,300,0.5098,0.8830,87.6,"(86.1, 89.0)"


In [12]:
import pandas as pd
model = 'densenet_BC_100_12_cm_1_1'
model_refined = model + '_refined400'

a = pd.Series([model, 400, 'cifar10', 0.4507, 0.9055])
c = pd.Series([model, 400, 'cifar10.1', 0.8656, 0.8250])

e = pd.Series([model_refined, 50, 'cifar10.1', 0.7624, 0.8645])
f = pd.Series([model_refined, 50, 'cifar10', 0.3269, 0.9369])
               
df_results = pd.concat([a,c,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 95.5 if row[2] == 'cifar10' else 87.6), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (95.1, 95.9) if row[2] == 'cifar10' else (86.1, 89.0)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/' + model + '/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,densenet_BC_100_12_cm_1_1,400,cifar10,0.4507,0.9055,95.5,"(95.1, 95.9)"
1,densenet_BC_100_12_cm_1_1,400,cifar10.1,0.8656,0.825,87.6,"(86.1, 89.0)"
2,densenet_BC_100_12_cm_1_1_refined400,50,cifar10.1,0.7624,0.8645,87.6,"(86.1, 89.0)"
3,densenet_BC_100_12_cm_1_1_refined400,50,cifar10,0.3269,0.9369,95.5,"(95.1, 95.9)"


In [13]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-june29'
prefix = 'sagemaker/results/original-models/densenet_BC_100_12_cm_1_1'
path = '/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_cm_1_1'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)